# Kaggle M5 Forecasting Model

In [ ]:
from datetime import datetime
import numpy as np
import pandas as pd
import dask.dataframe as dd
from catboost import CatBoostRegressor, Pool
import os
import pickle
import seaborn as sns
import matplotlib.pylab as plt
from pandas.plotting import autocorrelation_plot
import shap
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from tqdm import tqdm
import itertools
import multiprocessing

In [ ]:
pd.options.display.max_columns = 999

In [ ]:
def generate_pool(df, trial, subsample=True):
    if subsample:
        sample_df = df.sample(frac=trial["subsample"])
    else:
        sample_df = df
    return Pool(
        data=sample_df[trial["features"]], 
        label=sample_df["sales"],
        weight=sample_df["sell_price"],
        cat_features=trial["cat_features"],
    )

def generate_training_pools(train_splits, trial, subsample=True):
    return [generate_pool(train_splits[group], trial, subsample) for group in ("train", "eval", "test")]

def score_data(df, model, trail):
    pool = generate_pool(df, trail, subsample=False)
    df["prediction"] = model.predict(pool)
    df["total_sales_prediction"] = df["prediction"] * df["sell_price"]
    df["error"] = df["total_sales_prediction"] - df["total_sales"]
    df["abs_error"] = abs(df["total_sales_prediction"] - df["total_sales"])
    return df

## Load features

In [ ]:
def load_data(filename):
    input_df = pd.read_parquet(filename)
    for col in ("event_name_1", "event_type_1", "event_name_2", "event_type_2"):
        input_df[col] = input_df[col].cat.add_categories("NA").fillna("NA")
    train_splits = {group: input_df.query("group == @group") for group in ("train", "eval", "test", "forecast")}
    return train_splits

In [ ]:
train_splits = load_data("features/inputs.parquet")

In [ ]:
train_splits["train"].tail()

# Data exploration
- are there missing rows in the time series from when the product was not sold?
- are there missing or zero-valued prices?
- is the data zero-inflated?
- how to incorporate events?
- how to incorporate sales trends?
- how to incorporate pricing trends?

### Descriptive

In [ ]:
train_splits["train"].sample(frac=0.01).describe(percentiles=[0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99])

In [ ]:
train_splits["train"].sample(frac=0.01).select_dtypes(exclude=["number"]).describe()

### First entry in dataset

In [ ]:
sns.distplot(
    train_splits["train"].groupby("id")["day"].min()
)

In [ ]:
train_splits["train"].groupby("id")["day"].min().max()

## Incremental model builds

Model 1:
+ dates: dayofyear, year, month, dayofmonth, wday
+ product: dept, cat, item
+ store: store, state

Model 2:
+ rolling averages (7, 30, 121, 365 days; lagged by 1 day)

Model 3:
+ rolling averages (7, 30, 121, 365 days; lagged by 1, 7, 14, 28 days)

Model 4:
+ lag sales (1, 7, 14, 28 days)

Model 5:
+ weekday as categorical

Model 6:
+ month as categorical (no improvement)

Model 7:
+ change in sales (no improvement)

Model 8:
+ change in prices (no improvement)

Model 9:
+ snap eligible

Model 10:
+ test without lag features past 28 days

Model 11:
+ single store

In [ ]:
if os.path.isfile('data/experiments.pickle'):
    with open('data/experiments.pickle', 'rb') as handle:
        experiments = pickle.load(handle)
else:
    experiments = dict()

In [ ]:
experiments

In [ ]:
train_splits["train"].head()

In [ ]:
lag=28
trial = {
    "features": (
        ["dayofyear", "year", "month", "dayofmonth", "wday", "dept_id", "item_id", "store_id", "sell_price", f"sales_lag{lag}"]
        + [f"sales_lag{lag}_win{window}" for window in (30, 121, 365)]
    ),
    "cat_features": [
        "dept_id", "item_id", "store_id",
    ],
    "model_params": dict(
        iterations=100,
        boosting_type="Plain",
        loss_function="Tweedie:variance_power=1.1",
    ),
    "fit_params": dict(
        use_best_model=True,
        plot=True,
        metric_period=50,
    ),
    "subsample": 0.01,
}

In [ ]:
train_pool, eval_pool, test_pool = generate_training_pools(train_splits, trial)

In [ ]:
model = CatBoostRegressor(**trial["model_params"])

In [ ]:
model.fit(train_pool, eval_set=eval_pool, **trial["fit_params"])

current_time = datetime.now().strftime('%y%m%d%H%M%S')
trial["model_name"] = f"data/model_{current_time}.cbm"
model.save_model("data/model.cbm", pool=train_pool)
model.save_model(trial["model_name"], pool=train_pool)

In [ ]:
train_splits["eval"] = score_data(train_splits["eval"], model, trial)
score = lambda x: r2_score(x["sales"], x["prediction"], sample_weight=x["sell_price"])
trial["eval_score"] = score(train_splits["eval"])

In [ ]:
print(f"Eval score: {trial['eval_score']}")

## Feature importance

In [ ]:
shap_sample = train_splits["eval"].sample(frac=0.01)
shap_values = model.get_feature_importance(generate_pool(shap_sample, trial, subsample=False), type="ShapValues")
expected_value = shap_values[0,-1]
shap_values = shap_values[:,:-1]

In [ ]:
shap.summary_plot(shap_values, shap_sample[trial["features"]])

In [ ]:
shap.summary_plot(shap_values, shap_sample[trial["features"]], plot_type="bar")

## Save experiments

In [ ]:
trial_name = input("Enter trial name: ")
experiments[trial_name] = trial

In [ ]:
with open('data/experiments.pickle', 'wb') as handle:
    pickle.dump(experiments, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Visualize predictions

### Actuals

In [ ]:
sns.relplot(
    x="dayofyear",
    y="total_sales",
    hue="year",
    data=train_splits["eval"].groupby(["year", "dayofyear"], as_index=False)["total_sales"].sum(),
    kind="line",
    height=5,
    aspect=2,
)

### Prediction

In [ ]:
sns.relplot(
    x="dayofyear",
    y="total_sales_prediction",
    hue="year",
    data=train_splits["eval"].groupby(["year", "dayofyear"], as_index=False)["total_sales_prediction"].sum(),
    kind="line",
    height=5,
    aspect=2,
)

### Error

#### Bias

In [ ]:
plt.figure(figsize=(12, 6))
sns.distplot(np.minimum(25, np.maximum(-25, train_splits["eval"]["error"])))

In [ ]:
train_splits["eval"]["error"].describe()

#### Error autocorrelation

In [ ]:
def autocorr_plot_by_catid(df):
    plt.figure(figsize=(12, 6))
    ax = plt.gca(ylim=(-0.5, 0.5), xlim=(0, 25))
    autocorrelation_plot(df.groupby(["date"])["error"].mean().sort_index(), ax=ax)
    plt.title(df.cat_id.unique())
    plt.show()

In [ ]:
def autocorr_plot_by_store(df):
    plt.figure(figsize=(12, 6))
    ax = plt.gca(ylim=(-0.5, 0.5), xlim=(0, 25))
    autocorrelation_plot(df.groupby(["date"])["error"].mean().sort_index(), ax=ax)
    plt.title(df.store_id.unique())
    plt.show()

In [ ]:
train_splits["eval"].groupby(["cat_id"]).apply(lambda x: autocorr_plot_by_catid(x));

In [ ]:
train_splits["eval"].groupby(["store_id"]).apply(lambda x: autocorr_plot_by_store(x));

In [ ]:
plt.figure(figsize=(12, 6))
autocorrelation_plot(train_splits["eval"].groupby(["date"])["error"].mean().sort_index())
plt.ylim((-0.25,0.25))
plt.xlim((0,25))
plt.show();

#### Error over time

In [ ]:
sns.relplot(
    x="dayofyear",
    y="error",
    hue="year",
    data=train_splits["eval"].groupby(["year", "dayofyear"], as_index=False)["error"].mean(),
    kind="line",
    height=5,
    aspect=2,
)

In [ ]:
sns.relplot(
    x="dayofyear",
    y="error",
    hue="year",
    row="cat_id",
    data=train_splits["eval"].groupby(["year", "dayofyear", "cat_id"], as_index=False)["error"].mean(),
    kind="line",
    height=5,
    aspect=2,
)

#### Absolute error

In [ ]:
sns.relplot(
    x="dayofyear",
    y="abs_error",
    hue="year",
    data=train_splits["eval"].groupby(["year", "dayofyear"], as_index=False)["abs_error"].mean(),
    kind="line",
    height=5,
    aspect=2,
)

## Train day by day models

In [ ]:
models = {}

In [ ]:
for days_ahead in range(1, 28+1):
    trial = {
        "features": (
            [
                "dayofyear", "year", "month", "dayofmonth", "wday", "dept_id",
                "cat_id", "item_id", "store_id", "state_id", "snap",
                "event_name_1", "event_type_1", "event_name_2", "event_type_2",
            ]
            + [f"sales_lag{lag+days_ahead}_win{win}" for lag in (0, 7, 14, 28) for win in (7, 30, 121, 365)]
            + [f"sales_lag{lag+days_ahead}" for lag in (0, 7, 14, 28)]
            + [f"sell_price_lag{lag}_win{win}" for lag in (0, 7, 14, 28) for win in (7, 30, 121, 365)]
            + [f"sell_price_lag{lag}" for lag in (0, 7, 14, 28)]
        ),
        "cat_features": [
            "dept_id", "cat_id", "item_id", "store_id", "state_id", "wday",
            "event_name_1", "event_type_1", "event_name_2", "event_type_2",
        ],
        "model_params": dict(
            iterations=1000,
            boosting_type="Plain",
            loss_function="Tweedie:variance_power=1.1",
        ),
        "fit_params": dict(
            use_best_model=True,
            plot=True,
            metric_period=50,
        ),
        "subsample": 0.01,
    }
    
    train_pool, eval_pool, test_pool = generate_training_pools(train_splits, trial)
    
    model = CatBoostRegressor(**trial["model_params"])
    model.fit(train_pool, eval_set=eval_pool, **trial["fit_params"])
    models[days_ahead] = model
    
    train_splits["eval"] = score_data(train_splits["eval"], model, trial)
    trial["eval_score"] = r2_score(
        train_splits["eval"]["sales"],
        train_splits["eval"]["prediction"],
        sample_weight=train_splits["eval"]["sell_price"]
    )

    print(f"Eval score for day {days_ahead}: {trial['eval_score']}")
    
    predictions = model.predict(generate_pool(train_splits["forecast"], trial, subsample=False))
    train_splits["forecast"]["sales"] = np.where(
        train_splits["forecast"]["day"] == 1913 + days_ahead,
        predictions,
        train_splits["forecast"]["sales"]
    )

## Forecast

In [ ]:
train_splits["forecast"].tail()

In [ ]:
train_splits["forecast"]["submission_days_ahead"] = (train_splits["forecast"]["day"] - 1914) % 28 + 1
train_splits["forecast"]["type"] = (
    np.where((train_splits["forecast"]["day"]).gt(1913 + 28), "evaluation", "validation")
)

In [ ]:
train_splits["forecast"].groupby(["type"])["day"].nunique()

In [ ]:
submission_example = pd.read_csv("data/sample_submission.csv")

In [ ]:
submission_example.tail()

In [ ]:
submission = (
    train_splits["forecast"]
    .assign(f=lambda x: "F" + x["submission_days_ahead"].astype(str))
    .assign(id=lambda x: x["item_id"].astype(str) + "_" + x["store_id"].astype(str) + "_" + x["type"])
    .pivot(index="id", values="sales", columns="f")[[f"F{i+1}" for i in range(28)]]
    .fillna(0)
    .reset_index()
)

In [ ]:
submission.head()

In [ ]:
current_time = datetime.now().strftime('%y%m%d%H%M%S')
submission.to_csv("data/submission.csv", index=False)
submission.to_csv(f"data/submission_{current_time}", index=False)